In [71]:
import torch
from util import *
from dataset import *
from model import UNet
import torchvision.transforms as T
from torch.utils.data import DataLoader


In [72]:
png_dir = '/root/autoEncoder/result/png'
npy_dir = '/root/autoEncoder/result/numpy/'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [73]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.npy_dir = npy_dir
        lst_data = os.listdir(self.data_dir)

        lst_label_0 = [f for f in lst_data if f.startswith('label')]
        lst_input_0 = [f for f in lst_data if f.startswith('input')]
        lst_output_0 = [f for f in lst_data if f.startswith('output')]

        lst_npy = os.listdir(npy_dir)
        # print("numpy dir list %d" % len(lst_npy))
        # 넘파이 배열로 변환 후 저장
        if not lst_npy:
            for i, filename in enumerate(lst_label_0):
                name, ext = os.path.splitext(filename)
                if ext == ".jpg" or ext == ".png":
                    img_path = os.path.join(data_dir, filename)
                    img = Image.open(img_path)

                    # 이미지 리사이징을 하고 512X512 저장하기
                    transform = T.Resize((512, 512))
                    resize_img = transform(img)

                    np_path = os.path.join(npy_dir, name + ".npy")
                    np_img = np.asarray(resize_img)
                    np.save(np_path, np_img)

            for i, filename in enumerate(lst_input_0):
                name, ext = os.path.splitext(filename)
                if ext == ".jpg" or ext == ".png":
                    img = Image.open(os.path.join(data_dir, filename))
                    np_img = np.asarray(img)

                    # 이미지 리사이징을 하고 512X512 저장하기
                    # transform = T.Resize((512, 512))
                    # resize_img = transform(img)

                    np_path = os.path.join(npy_dir, name + ".npy")
                    np_img = np.asarray(resize_img)
                    np.save(np_path, np_img)
                    
            for i, filename in enumerate(lst_output_0):
                name, ext = os.path.splitext(filename)
                if ext == ".jpg" or ext == ".png":
                    img = Image.open(os.path.join(data_dir, filename))
                    np_img = np.asarray(img)

                    # 이미지 리사이징을 하고 512X512 저장하기
                    transform = T.Resize((512, 512))
                    resize_img = transform(img)

                    np_path = os.path.join(npy_dir, name + ".npy")
                    print(np_path)
                    np_img = np.asarray(resize_img)
                    np.save(np_path, np_img)


        lst_npy = os.listdir(npy_dir)
        
        lst_label = [f for f in lst_npy if f.startswith(
            'label') and f.endswith('.npy')]
        lst_input = [f for f in lst_npy if f.startswith(
            'input') and f.endswith('.npy')]
        lst_output = [f for f in lst_npy if f.startswith(
            'output') and f.endswith('.npy')]
        # lst_label = [f for f in lst_data if f.startswith('label')]
        # lst_input = [f for f in lst_data if f.startswith('input')]

        lst_label.sort()
        lst_input.sort()
        lst_output.sort()
 
        self.lst_label = lst_label
        self.lst_input = lst_input
        self.lst_output = lst_output

    def __len__(self):
        return len(self.lst_label)

    def __getitem__(self, index):
        label = np.load(os.path.join(self.npy_dir, self.lst_label[index]))
        input = np.load(os.path.join(self.npy_dir, self.lst_input[index]))
        output = np.load(os.path.join(self.npy_dir, self.lst_output[index]))

        label = label/255.0
        input = input/255.0
        output = output/255.0

        if label.ndim == 2:
            label = label[:, :, np.newaxis]
        if input.ndim == 2:
            input = input[:, :, np.newaxis]
        if output.ndim == 2:
            output = output[:, :, np.newaxis]

        data = {'input': input, 'label': label, 'output': output}

        if self.transform:
            data = self.transform(data)

        return data
    
# 트렌스폼 구현하기
class ToTensor(object):
    def __call__(self, data):
        label, input, output = data['label'], data['input'], data['output']

        label = label.transpose((2, 0, 1)).astype(np.float32)
        input = input.transpose((2, 0, 1)).astype(np.float32)
        output = output.transpose((2, 0, 1)).astype(np.float32)

        data = {'label': torch.from_numpy(
            label), 'input': torch.from_numpy(input), 'output': torch.from_numpy(output)}

        return data


In [74]:
EPS = 1e-8


def PSNR(input, target):
    return -10*torch.log10(torch.mean((input - target) ** 2, dim=[1, 2, 3])+EPS)


def MSE(input, target):
    return torch.mean((input - target) ** 2, dim=[1, 2, 3])


In [75]:
dataset_result = Dataset(data_dir=png_dir, transform=ToTensor())
loader_result = DataLoader(dataset_result, batch_size=1, shuffle=False, num_workers=0)
print(dataset_result.lst_input)
print(dataset_result.lst_label)
print(dataset_result.lst_output)


/root/autoEncoder/result/numpy/output_0070.npy
/root/autoEncoder/result/numpy/output_0010.npy
/root/autoEncoder/result/numpy/output_0061.npy
/root/autoEncoder/result/numpy/output_0012.npy
/root/autoEncoder/result/numpy/output_0052.npy
/root/autoEncoder/result/numpy/output_0031.npy
/root/autoEncoder/result/numpy/output_0072.npy
/root/autoEncoder/result/numpy/output_0081.npy
/root/autoEncoder/result/numpy/output_0090.npy
/root/autoEncoder/result/numpy/output_0033.npy
/root/autoEncoder/result/numpy/output_0032.npy
/root/autoEncoder/result/numpy/output_0062.npy
/root/autoEncoder/result/numpy/output_0073.npy
/root/autoEncoder/result/numpy/output_0083.npy
/root/autoEncoder/result/numpy/output_0051.npy
/root/autoEncoder/result/numpy/output_0043.npy
/root/autoEncoder/result/numpy/output_0003.npy
/root/autoEncoder/result/numpy/output_0001.npy
/root/autoEncoder/result/numpy/output_0000.npy
/root/autoEncoder/result/numpy/output_0023.npy
/root/autoEncoder/result/numpy/output_0092.npy
/root/autoEnc

In [76]:
psnr = []
mse = []
for batch, data in enumerate(loader_result, 1):
    label = data['label']
    input = data['input']
    output = data['output']
    
    print("PSNR : " + str(PSNR(label, output)))
    print("MSE : " + str(PSNR(label, output)))
    psnr.extend(PSNR(label, output))
    mse.extend(MSE(label, output))


PSNR : tensor([24.9071])
MSE : tensor([24.9071])
PSNR : tensor([35.2846])
MSE : tensor([35.2846])
PSNR : tensor([27.7550])
MSE : tensor([27.7550])
PSNR : tensor([25.0061])
MSE : tensor([25.0061])
PSNR : tensor([24.5719])
MSE : tensor([24.5719])
PSNR : tensor([27.7129])
MSE : tensor([27.7129])
PSNR : tensor([26.3763])
MSE : tensor([26.3763])
PSNR : tensor([20.7844])
MSE : tensor([20.7844])
PSNR : tensor([24.9427])
MSE : tensor([24.9427])
PSNR : tensor([30.4596])
MSE : tensor([30.4596])
PSNR : tensor([25.3081])
MSE : tensor([25.3081])
PSNR : tensor([25.7235])
MSE : tensor([25.7235])
PSNR : tensor([19.9171])
MSE : tensor([19.9171])
PSNR : tensor([29.3417])
MSE : tensor([29.3417])
PSNR : tensor([26.8808])
MSE : tensor([26.8808])
PSNR : tensor([28.2049])
MSE : tensor([28.2049])
PSNR : tensor([32.6763])
MSE : tensor([32.6763])
PSNR : tensor([24.8544])
MSE : tensor([24.8544])
PSNR : tensor([27.6142])
MSE : tensor([27.6142])
PSNR : tensor([26.7232])
MSE : tensor([26.7232])
PSNR : tensor([30.39

In [77]:
mse_psnr = psnr.copy()
print(len(mse_psnr))
mse_psnr = np.array(mse_psnr)
print(
    f"The mean of the PSNR is {mse_psnr.mean()} and the standard deviation of the PSNR is {mse_psnr.std()}")


40
The mean of the PSNR is 26.38802146911621 and the standard deviation of the PSNR is 3.0864531993865967


In [78]:
mse = mse.copy()
print(len(mse))
mse = np.array(mse)
print(
    f"The mean of the MSE is {mse.mean()} and the standard deviation of the MSE is {mse.std()}")


40
The mean of the MSE is 0.0029156599193811417 and the standard deviation of the MSE is 0.002142836106941104
